In [9]:
import pandas as pd
import numpy as np

import visualizer as vs
import convolutional_neural_network as cnn

In [10]:
data = pd.read_csv("ORIGINAL_TRAINING_DATA/GRADIENTS/data1.csv")
array, headers = cnn.data_arranger(data, [200,200])

Normalizing gradient datas...
Rearranging data to tensor for model classification...
Obtained regularized tensor.


In [11]:
array

array([[[2.09833997e-01, 2.06857406e-01, 2.01575755e-01, ...,
         4.05331830e-01,            nan,            nan],
        [2.11450798e-01, 2.08509303e-01, 2.03703786e-01, ...,
         4.17559762e-01,            nan,            nan],
        [2.13258443e-01, 2.12125067e-01, 2.07482178e-01, ...,
         4.51766337e-01,            nan,            nan],
        ...,
        [3.70074197e-01, 3.62557606e-01, 3.42386091e-01, ...,
         3.71420416e-01,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]],

       [[0.00000000e+00, 1.00877964e-01, 1.41926021e-01, ...,
         2.20613803e-01,            nan,            nan],
        [1.02222339e-01, 9.19552343e-02, 1.22940707e-01, ...,
         1.81398384e-01,            nan,            nan],
        [1.47302387e-01, 

In [13]:
test_input = np.array([
    [
        [0.4, 0.1],
        [0.8, 0.2]
    ],
    [
        [0.4, 0.8],
        [0.1, 0.9]
    ],
    [
        [0.8, 0.2],
        [0.5, 0.6]
    ]
])
test_classification =  np.array([
    [0.2, 0.1],
    [0.7, 0.9]
])
header_list = ['temperature_gradient', 'velocity_magnitude_gradient','z_velocity_gradient']

In [16]:
loss = cnn.loss_function(test_input,header_list, test_classification)

AttributeError: 'list' object has no attribute 'shape'